In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras


from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
# y_train = min_max_scaler.fit_transform(y_train)


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [19]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences

In [20]:
timesteps 

184

In [21]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [22]:
X_train.shape

(3499, 2, 1, 184, 1)

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=20)

In [24]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=16, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss='mse')
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=500, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/500
3499/3499 [==============================] - 7s 2ms/step - loss: 10359.5778 - val_loss: 9932.8272
Epoch 2/500
3499/3499 [==============================] - 6s 2ms/step - loss: 10330.7143 - val_loss: 9884.0376
Epoch 3/500
3499/3499 [==============================] - 6s 2ms/step - loss: 10258.8020 - val_loss: 9777.1923
Epoch 4/500
3499/3499 [==============================] - 6s 2ms/step - loss: 10111.2145 - val_loss: 9569.6503
Epoch 5/500
3499/3499 [==============================] - 6s 2ms/step - loss: 9832.4343 - val_loss: 9187.3444
Epoch 6/500
3499/3499 [==============================] - 6s 2ms/step - loss: 9327.5704 - val_loss: 8508.6025
Epoch 7/500
3499/3499 [==============================] - 6s 2ms/step - loss: 8441.7870 - val_loss: 7337.4965
Epoch 8/500
3499/3499 [==============================] - 6s 2ms/step - loss: 6937.0674 - val_loss: 5398.9394
Epoch 9/500
3499/3499 [==============================] - 6s 2ms/step - loss: 

3499/3499 [==============================] - 10s 3ms/step - loss: 160.6477 - val_loss: 126.8497
Epoch 76/500
3499/3499 [==============================] - 10s 3ms/step - loss: 160.5400 - val_loss: 126.6094
Epoch 77/500
3499/3499 [==============================] - 10s 3ms/step - loss: 160.1961 - val_loss: 125.4206
Epoch 78/500
3499/3499 [==============================] - 11s 3ms/step - loss: 160.0020 - val_loss: 126.9901
Epoch 79/500
3499/3499 [==============================] - 11s 3ms/step - loss: 159.8956 - val_loss: 126.2280
Epoch 80/500
3499/3499 [==============================] - 10s 3ms/step - loss: 159.6096 - val_loss: 126.2601
Epoch 81/500
3499/3499 [==============================] - 10s 3ms/step - loss: 159.4248 - val_loss: 125.6298
Epoch 82/500
3499/3499 [==============================] - 11s 3ms/step - loss: 159.1829 - val_loss: 125.1721
Epoch 83/500
3499/3499 [==============================] - 10s 3ms/step - loss: 159.0043 - val_loss: 125.7331
Epoch 84/500
3499/3499 [========

Epoch 150/500
3499/3499 [==============================] - 10s 3ms/step - loss: 142.5467 - val_loss: 106.0977
Epoch 151/500
3499/3499 [==============================] - 10s 3ms/step - loss: 141.3936 - val_loss: 108.6523
Epoch 152/500
3499/3499 [==============================] - 10s 3ms/step - loss: 140.3931 - val_loss: 106.5143
Epoch 153/500
3499/3499 [==============================] - 10s 3ms/step - loss: 139.8126 - val_loss: 106.0444
Epoch 154/500
3499/3499 [==============================] - 11s 3ms/step - loss: 139.4502 - val_loss: 107.1948
Epoch 155/500
3499/3499 [==============================] - 11s 3ms/step - loss: 138.5739 - val_loss: 103.3919
Epoch 156/500
3499/3499 [==============================] - 10s 3ms/step - loss: 138.0943 - val_loss: 107.2041
Epoch 157/500
3499/3499 [==============================] - 10s 3ms/step - loss: 137.5152 - val_loss: 101.6259
Epoch 158/500
3499/3499 [==============================] - 10s 3ms/step - loss: 136.9087 - val_loss: 102.4469
Epoch 159/

3499/3499 [==============================] - 10s 3ms/step - loss: 81.0750 - val_loss: 53.6068
Epoch 226/500
3499/3499 [==============================] - 10s 3ms/step - loss: 81.1131 - val_loss: 51.0887
Epoch 227/500
3499/3499 [==============================] - 10s 3ms/step - loss: 81.2372 - val_loss: 54.1423
Epoch 228/500
3499/3499 [==============================] - 11s 3ms/step - loss: 81.9272 - val_loss: 57.7829
Epoch 229/500
3499/3499 [==============================] - 11s 3ms/step - loss: 81.2721 - val_loss: 60.7352
Epoch 230/500
3499/3499 [==============================] - 11s 3ms/step - loss: 81.4656 - val_loss: 58.8353
Epoch 231/500
3499/3499 [==============================] - 10s 3ms/step - loss: 81.3856 - val_loss: 51.0603
Epoch 232/500
3499/3499 [==============================] - 11s 3ms/step - loss: 81.5274 - val_loss: 49.4232
Epoch 233/500
3499/3499 [==============================] - 10s 3ms/step - loss: 81.4470 - val_loss: 47.9881
Epoch 234/500
3499/3499 [=================

Epoch 301/500
3499/3499 [==============================] - 10s 3ms/step - loss: 78.7181 - val_loss: 52.6787
Epoch 302/500
3499/3499 [==============================] - 10s 3ms/step - loss: 78.7540 - val_loss: 57.1292
Epoch 303/500
3499/3499 [==============================] - 10s 3ms/step - loss: 78.9642 - val_loss: 59.3592
Epoch 304/500
3499/3499 [==============================] - 10s 3ms/step - loss: 79.8691 - val_loss: 61.1157
Epoch 305/500
3499/3499 [==============================] - 10s 3ms/step - loss: 79.4882 - val_loss: 61.0357
Epoch 306/500
3499/3499 [==============================] - 10s 3ms/step - loss: 79.0053 - val_loss: 51.8686
Epoch 307/500
3499/3499 [==============================] - 10s 3ms/step - loss: 78.5575 - val_loss: 54.2397
Epoch 308/500
3499/3499 [==============================] - 10s 3ms/step - loss: 78.9572 - val_loss: 55.3365
Epoch 309/500
3499/3499 [==============================] - 10s 3ms/step - loss: 78.4609 - val_loss: 52.1010
Epoch 310/500
3499/3499 [===

Epoch 377/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.5187 - val_loss: 53.6481
Epoch 378/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.9714 - val_loss: 50.4530
Epoch 379/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.7924 - val_loss: 51.4608
Epoch 380/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.4366 - val_loss: 51.9300
Epoch 381/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.3015 - val_loss: 47.3549
Epoch 382/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.6921 - val_loss: 52.1044
Epoch 383/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.6277 - val_loss: 53.5123
Epoch 384/500
3499/3499 [==============================] - 10s 3ms/step - loss: 77.3260 - val_loss: 50.7087
Epoch 385/500
3499/3499 [==============================] - 11s 3ms/step - loss: 77.1884 - val_loss: 52.3662
Epoch 386/500
3499/3499 [===

3499/3499 [==============================] - 6s 2ms/step - loss: 76.6251 - val_loss: 53.9841
Epoch 454/500
3499/3499 [==============================] - 6s 2ms/step - loss: 76.9345 - val_loss: 54.7732
Epoch 455/500
3499/3499 [==============================] - 6s 2ms/step - loss: 76.9898 - val_loss: 57.9814
Epoch 456/500
3499/3499 [==============================] - 6s 2ms/step - loss: 76.6846 - val_loss: 52.8662
Epoch 457/500
3499/3499 [==============================] - 6s 2ms/step - loss: 76.9377 - val_loss: 55.0368
Epoch 458/500
3499/3499 [==============================] - 6s 2ms/step - loss: 77.1917 - val_loss: 54.2735
Epoch 459/500
3499/3499 [==============================] - 6s 2ms/step - loss: 77.1816 - val_loss: 53.1715
Epoch 460/500
3499/3499 [==============================] - 6s 2ms/step - loss: 76.5432 - val_loss: 51.3700
Epoch 461/500
3499/3499 [==============================] - 6s 2ms/step - loss: 76.9034 - val_loss: 50.8554
Epoch 462/500
3499/3499 [==========================

In [25]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.6574914445346058,
 'Max Error': 39.8500033569336,
 'Mean Absolute Error': 6.030188689151053,
 'Mean Squared Error': 54.8335854195177,
 'Root Mean Squared Error': 7.404970318611527,
 'Median Absolute Error': 5.491190490722651,
 'R² Score': 0.5836907923261785,
 'Mean Absolute Percentage Error': 5.900770355168048}

In [26]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
